In [28]:
import glob
import lizard
import json
import csv
import os
import re
import pickle
from os import path

In [29]:
from prep_for_headers import prep
from prep_for_headers import prep_for_macros
from getusefullheaders import find_all_used_header 
from comment_remover import comment_remover

In [30]:
cnt = 0
lenn = 0
pickle_list = []
lenlol = []

for file in glob.glob("*.cpp"):
        lenn = lenn + 1

In [31]:
def fin(st, code):
    d = re.findall(st, code)
    return len(d)

In [32]:
def datatype_info(data):
    d = {}
    usli = fin('unsigned long long int',data)
    usl = fin('unsigned long long',data) - usli
    d["unsigned ll"] = usl + usli
    unsi = fin('unsigned int',data)
    d["unisgned int"] = unsi
    unsili = fin('unsigned long int',data)
    unsil = fin('unsigned long',data) - (usli + usl + unsili)
    d["unsigned long"] = unsili + unsil
    lli = fin('long long int',data) - usli
    ll =  fin('long long',data) - (usl + usli + lli)
    d["long long"] = lli + ll
    li =  fin('long int',data) - (lli + usli + unsili)
    l = fin('long',data) - (2*usli + 2*usl + unsili + unsil + 2*lli + 2*ll + li)
    d["long"] = li + l
    i =   fin('int',data) - (usli + lli + li + unsili + unsi) - fin('int main',data)
    d["int"] = i
    ldb = fin('long double',data)
    d["long double"] = ldb
    db = fin('double',data) - ldb
    d["double"] = db
    fl = fin('float',data)
    d["float"] = fl
    char = fin('char',data)
    d["char"] = char
    string = fin('string',data)
    d["string"] = string
    sc = fin('scanf',data)
    d["scanf"] = sc
    pr = fin('printf',data)
    d["printf"] = pr
    cin = fin('cin',data)
    d["cin"] = cin
    cout = fin('cout',data)
    d["cout"] = cout
    getchar = fin('getchar',data)
    d["getchar"] = getchar
    defi = len([x.start() for x in re.finditer('#define', data)])
    d["#define"] = defi
    typed = len([x.start() for x in re.finditer('typedef', data)])
    d["typedef"] = typed
    prag = fin('#pragma',data)
    if prag>0:
        d["#pragma"] = 1
    else:
        d["#pragma"] = 0
    fast = fin('ios_base',data)
    if fast>0:
        d["fast_io"] = 1
    else:
        d["fast_io"] = 0
    return d

In [33]:
def op_info(code):
    d = {}
    d["arithmetic_operators"] = code.count(
        '+') + code.count('-') + code.count('*') + code.count('/')
    noOfAnd = code.count('&&') + code.count(' and ')
    noOfOr = code.count('||') + code.count(' or ')
    noOfNot = code.count('!')
    noOfBAnd = code.count('&') - (code.count('&&')*2)
    noOfBOr = code.count('|') - (code.count('||')*2)
    d["boolean_operators"] = noOfAnd + noOfOr
    d["binary_operators"] = noOfBAnd + noOfBOr + noOfNot
    return d

In [34]:
while True:
    try:
        try:
            with open('pickle_list.pkl', 'rb') as f:
                pickle_list = pickle.load(f)
                cnt = len(pickle_list)
        except:
            pass

        if lenn == len(pickle_list):
            break
        if len(pickle_list) > 0:
            mode = 'a'
        else:
            mode = 'w'

        with open("out_master.json", mode) as file_handler:

            if len(pickle_list) == 0:
                file_handler.write("[")

            ct = 1

            for file in glob.glob("*.cpp"):

                if file in pickle_list:
                    continue
                cnt = cnt + 1
                print(cnt, file)

                i = lizard.analyze_file(file)
                ob = i.__dict__
                if len(ob['function_list']) == 0:
                    continue

                #print (i.function_list[0].__dict__)
                json_data = open('./' + file.partition(".")[0] + '.json')
                data = json.load(json_data)
                json_data.close()

                d = {}
                d["filename"] = ob['filename']
                d["nloc"] = ob['nloc']
                d["token_count"] = ob['token_count']
                d["problem"] = data['problemlink']
                d["solution"] = data['sollink']
                d["exec_time"] = ' '.join(data['exec_time'].split())
                d["memory"] = ' '.join(data['memory'].split())
                d["usertype"] = data["usertype"]
                d["function_count"] = len(ob['function_list'])

                c = 0
                mc = 0
                used_headers = 0
                used_macros = 0
                vec_ct = 0
                list_ct = 0
                pbds = 0
                map_ct = 0
                umap_ct = 0

                with open(file, 'r') as f:
                    #f.sub('//.*?\n|/\*.*?\*/', '', text, flags=re.S)
                    # print(f.read())
                    f = comment_remover(f.read())
                    
                    dt_res = datatype_info(f)
                    op_res = op_info(f)

                    header_list = prep(f)
                    # print(header_list)
                    macro_list = prep_for_macros(f)
                    x = find_all_used_header(f, header_list[0], header_list[1])
                    # print(x)
                    if len(macro_list[0]) > 0:
                        # print(macro_list[0])
                        # print(macro_list[1])
                        macrox = find_all_used_header(
                            f, macro_list[0], macro_list[1])
                        # print(macrox)
                        for key, val in macrox.items():
                            if val == 1:
                                used_macros += 1

                    for key, val in x.items():
                        if val == 1:
                            used_headers += 1

                    for line in f.split('\n'):
                        #line = line.strip()
                        if '#include' in line:
                            c += 1
                        if '#define' in line or 'typedef' in line:
                            mc += 1
                        if 'vector' in line:
                            vec_ct += 1
                        if 'list' in line:
                            list_ct += 1
                        if '__gnu_pbds' in line:
                            pbds += 1
                        if 'map' in line:
                            map_ct += 1
                        if 'unordered_map' in line:
                            umap_ct += 1
                            map_ct -= 1
                    
                d["specific_counts"] = {
                    "vector": vec_ct,
                    "list": list_ct,
                    "pbds": pbds,
                    "map": map_ct,
                    "unordered_map": umap_ct,
                }
                
                d["datatype_counts"] = dt_res
                d["operator_counts"] = op_res

                # Take c value here
                d["library_count"] = c
                d["used_headers"] = used_headers
                # print(mc)
                d["macro_count"] = mc
                d["used_macros"] = used_macros

                c = 0
                fd = []
                ct = 0
                for func in ob['function_list']:
                    fn = i.function_list[ct].__dict__
                    ct = ct + 1
                    fd.append(fn)

                d["function_details"] = fd
                if(len(d['usertype']) == 2):
                    d['usertype'] = 'us'
                item = json.dumps(d, ensure_ascii=False)
                # print(jsonob)

                try:
                    if cnt < lenn:
                        file_handler.write("{},\n".format(item))
                    else:
                        file_handler.write("{}\n".format(item))
                except:
                    print(len(d['usertype']))
                    exit(-1)

                pickle_list.append(file)
            
            file_handler.write("]")
            with open('pickle_list.pkl', 'wb') as f:
                pickle.dump(pickle_list, f)
            break

    except Exception as e:

        pickle_list.append(file)
        if str(e) == 'len(headers) != len(positions but expected to be of same length':
            lenlol.append(file)
        with open('pickle_list.pkl', 'wb') as f:
            pickle.dump(pickle_list, f)


1 1.cpp
2 2.cpp
3 3.cpp


## Sample JSON Output for a file

```
[{
    "filename": "1.cpp",
    "nloc": 45,
    "token_count": 296,
    "problem": "https://www.codeforces.com/contest/492/problem/B",
    "solution": "https://www.codeforces.com/contest/492/submission/44082853",
    "exec_time": "31 ms",
    "memory": "200 KB",
    "usertype": "Expert",
    "function_count": 2,
    "specific_counts": {
        "vector": 1,
        "list": 0,
        "pbds": 0,
        "map": 0,
        "unordered_map": 0
    },
    "datatype_counts": {
        "unsigned ll": 0,
        "unisgned int": 0,
        "unsigned long": 0,
        "long long": 0,
        "long": 0,
        "int": 0,
        "long double": 0,
        "double": 7,
        "float": 0,
        "char": 0,
        "string": 0,
        "scanf": 0,
        "printf": 0,
        "cin": 4,
        "cout": 4,
        "getchar": 0,
        "#define": 0,
        "typedef": 0,
        "#pragma": 0,
        "fast_io": 0
    },
    "operator_counts": {
        "arithmetic_operators": 13,
        "boolean_operators": 0,
        "binary_operators": 3
    },
    "library_count": 1,
    "used_headers": 1,
    "macro_count": 0,
    "used_macros": 0,
    "function_details": [
        {
            "cyclomatic_complexity": 6,
            "nloc": 31,
            "token_count": 215,
            "name": "solve",
            "long_name": "solve()",
            "start_line": 13,
            "end_line": 83,
            "parameters": [],
            "filename": "1.cpp",
            "top_nesting_level": 0,
            "length": 71,
            "fan_in": 0,
            "fan_out": 0,
            "general_fan_out": 0
        },
        {
            "cyclomatic_complexity": 3,
            "nloc": 10,
            "token_count": 62,
            "name": "main",
            "long_name": "main()",
            "start_line": 87,
            "end_line": 109,
            "parameters": [],
            "filename": "1.cpp",
            "top_nesting_level": 0,
            "length": 23,
            "fan_in": 0,
            "fan_out": 0,
            "general_fan_out": 0
        }
    ]
}]
```